In [51]:
import numpy as np
from pymongo import MongoClient
import pydicom as pyd
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import scipy.misc
import xml.etree.ElementTree as ET
import os
from random import shuffle
from shutil import copyfile

In [2]:
#Connecting to database
client = MongoClient('localhost', 27017)
db = client['rsna-xray']
col = db['patient']

In [3]:
#Getting the objects with lung opacities
lung_opacities = col.find({"class":"Lung Opacity"})
#Converting to list
lung_opacities = list(lung_opacities)

In [31]:
data = ET.Element('annotation')
folder = ET.SubElement(data, 'folder')
folder.text = "Final Project"
file = ET.SubElement(data, "filename")
file.text = "image.jpg"
path = ET.SubElement(data, "path")
path.text = os.getcwd() + "/{:s}".format("image.jpg")
source = ET.SubElement(data, "source")
datab = ET.SubElement(source, "database")
datab.text = "Unknown"
size = ET.SubElement(data, "size")
width = ET.SubElement(size, "width")
height = ET.SubElement(size, "height")
depth = ET.SubElement(size, "depth")
width.text = "1024"
height.text = "1024"
depth.text = "1"
seg = ET.SubElement(data, "segmented")
seg.text = "0"
obj = ET.SubElement(data, "object")
name1 = ET.SubElement(obj, "name")
name1.text = "lungopacity"
pose1 = ET.SubElement(obj, "pose")
pose1.text = "Unspecified"
truncated = ET.SubElement(obj, "truncated")
truncated.text = "0"
diff = ET.SubElement(obj, "difficult")
diff.text = "0"
bndbox = ET.SubElement(obj, "bndbox")
xmin = ET.SubElement(bndbox, "xmin")
xmin.text = "570"
ymin = ET.SubElement(bndbox, "ymin")
ymin.text = "262"
xmax = ET.SubElement(bndbox, "xmax")
xmax.text = "670"
ymax = ET.SubElement(bndbox, "ymax")
ymax.text = "551"
obj = ET.SubElement(data, "object")
name1 = ET.SubElement(obj, "name")
name1.text = "lungopacity"
pose1 = ET.SubElement(obj, "pose")
pose1.text = "Unspecified"
truncated = ET.SubElement(obj, "truncated")
truncated.text = "0"
diff = ET.SubElement(obj, "difficult")
diff.text = "0"
bndbox = ET.SubElement(obj, "bndbox")
xmin = ET.SubElement(bndbox, "xmin")
xmin.text = "570"
ymin = ET.SubElement(bndbox, "ymin")
ymin.text = "262"
xmax = ET.SubElement(bndbox, "xmax")
xmax.text = "670"
ymax = ET.SubElement(bndbox, "ymax")
ymax.text = "551"

In [32]:
mydata = ET.tostring(data, encoding="unicode")
with open("img.xml", "w") as f:
    f.write(mydata)
    f.close()

In [27]:
mydata

b'<annotation><folder>Final Project</folder><filename>image.jpg</filename><path>/Users/alexhyman/Documents/Syracuse/Current Classes/Big Data Analytics/Project/Final Project/image.jpg</path><source><database>Unknown</database></source><size><width>1024</width><height>1024</height><depth>1</depth></size><segmented>0</segmented><object><name>lungopacity</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>570</xmin><ymin>262</ymin><xmax>670</xmax><ymax>551</ymax></bndbox></object></annotation>'

In [13]:
def img_xml_convert(doc, folderpath = ""):
    #read the file
    f = pyd.read_file(doc["path"])
    #Get the image
    img = f.pixel_array
    #save the picture
    scipy.misc.imsave(folderpath + doc["patientID"] + ".jpg", img)
    
    #creating the xml
    data = ET.Element('annotation')
    folder = ET.SubElement(data, 'folder')
    if folderpath == "":
        folder.text = os.path.basename(os.getcwd())
    else:
        folder.text = folderpath
    file = ET.SubElement(data, "filename")
    file.text = folderpath + doc["patientID"] + ".jpg"
    path = ET.SubElement(data, "path")
    if folderpath == "":
        path.text = os.getcwd() + "/{:s}.jpg".format(doc["patientID"])
    else:
        path.text = os.getcwd() + "/{:s}/{:s}.jpg".format(folderpath, doc["patientID"])
    source = ET.SubElement(data, "source")
    datab = ET.SubElement(source, "database")
    datab.text = "Unknown"
    size = ET.SubElement(data, "size")
    width = ET.SubElement(size, "width")
    height = ET.SubElement(size, "height")
    depth = ET.SubElement(size, "depth")
    width.text = str(img.shape[1])
    height.text = str(img.shape[0])
    depth.text = "1"
    seg = ET.SubElement(data, "segmented")
    seg.text = "0"
    for target in doc["targets"]:
        obj = ET.SubElement(data, "object")
        name = ET.SubElement(obj, "name")
        name.text = "lungopacity"
        pose = ET.SubElement(obj, "pose")
        pose.text = "Unspecified"
        truncated = ET.SubElement(obj, "truncated")
        truncated.text = "0"
        diff = ET.SubElement(obj, "difficult")
        diff.text = "0"
        bndbox = ET.SubElement(obj, "bndbox")
        xmin = ET.SubElement(bndbox, "xmin")
        xmin.text = str(int(target[0]))
        ymin = ET.SubElement(bndbox, "ymin")
        ymin.text = str(int(target[1]))
        xmax = ET.SubElement(bndbox, "xmax")
        xmax.text = str(int(target[0] + target[2]))
        ymax = ET.SubElement(bndbox, "ymax")
        ymax.text = str(int(target[1] + target[3]))
    strdata = ET.tostring(data, encoding="unicode")
    with open("{:s}{:s}.xml".format(folderpath, doc["patientID"]), "w") as f:
        f.write(strdata)
        f.close()

In [9]:
#Testing function
img_xml_convert(lung_opacities[0])
img_xml_convert(lung_opacities[2])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  import sys


In [11]:
shuffle(lung_opacities)

In [15]:
img_xml_convert(lung_opacities[0], "object-detection/images/")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  import sys


In [16]:
for doc in lung_opacities:
    img_xml_convert(doc, "object-detection/images/")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  import sys


In [17]:
os.chdir("object-detection/images/")

In [45]:
import re

files = os.listdir()
files.remove("train")
files.remove("test")
filenames = [f[:-4] for f in files]
filenames = list(set(filenames))
ix = np.arange(len(filenames)).tolist()
shuffle(ix)

In [54]:
split_ix = int(np.floor(0.8 * len(ix)))
train_ix = ix[:split_ix]
test_ix = ix[split_ix:]

In [55]:
train_files = [(filenames[i] + ".jpg", filenames[i] + ".xml") for i in train_ix]
test_files = [(filenames[i] + ".jpg", filenames[i] + ".xml") for i in test_ix]

In [60]:
for (jpg, xml) in train_files:
    copyfile(jpg, "train/"+ jpg)
    copyfile(xml, "train/"+ xml)

In [61]:
for (jpg, xml) in test_files:
    copyfile(jpg, "test/"+ jpg)
    copyfile(xml, "test/"+ xml)